In [ ]:
import pandas as pd
import numpy as np


Mounted at /content/drive


In [ ]:
#import data
df = pd.read_csv('df_ada_embedded_floats_CC1204.csv')

In [ ]:
y = df['BloodGlucose']
data2 = df.iloc[:,:17]
df_embedded = df.iloc[:,1:]
embeddings = df.iloc[:,18:]

In [ ]:
df.iloc[:,17:].head(2)

,BloodGlucose,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,78.110787,-0.016344,0.027581,0.027481,0.253978,0.418981,-0.251701,0.181153,0.253314,-0.034449,...,0.181150,0.053004,-0.392427,-0.131862,-0.092827,0.216697,0.146793,0.021225,0.060077,-0.121716
1,55.974437,-0.024589,0.011475,0.021612,0.258093,0.415543,-0.256149,0.172979,0.243002,-0.028191,...,0.193478,0.072145,-0.400099,-0.133023,-0.104748,0.203521,0.155139,-0.004050,0.032071,-0.124583


The following chunk of code initializes a DML model that includes both the OUTCOME and the TREATMENT regression.

In [ ]:
pip install econml

In [ ]:
import xgboost
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from econml.dml import DML
from sklearn.model_selection import KFold

# Standardize Data

In [ ]:
y = df['BloodGlucose']
T = df['Treatment']
df2 = df.copy()
df2.drop(columns=['Treatment', 'BloodGlucose'], inplace = True)
data2 = df.iloc[:, 1:16]
embeddings = df.iloc[:, 16:]

from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Standardize tabular data (data2)
scaler = StandardScaler()
tab_data = scaler.fit_transform(data2)

# L2 normalization for embeddings
embedding_magnitudes = np.linalg.norm(embeddings, axis=1, keepdims=True) + 1e-8
normalized_embeddings = embeddings / embedding_magnitudes



# non
df_none = pd.concat([pd.DataFrame(data2, columns=data2.columns),
                           pd.DataFrame(embeddings, columns=embeddings.columns)], axis=1)


# Combine standardized tabular data and normalized embeddings
df_embedded = pd.concat([pd.DataFrame(tab_data, columns=data2.columns),
                           pd.DataFrame(normalized_embeddings, columns=embeddings.columns)], axis=1)




# Standardize embeddings

standardized_embeddings = scaler.fit_transform(embeddings)
# Combine standardized tabular data and normalized embeddings
df_embedded_std = pd.concat([pd.DataFrame(tab_data, columns=data2.columns),
                           pd.DataFrame(standardized_embeddings, columns=embeddings.columns)], axis=1)


# Full

# Combine standardized tabular data and normalized embeddings
df_embedded_full = pd.concat([pd.DataFrame(tab_data, columns=data2.columns),
                           pd.DataFrame(embeddings, columns=embeddings.columns)], axis=1)


tb = pd.DataFrame(tab_data, columns=data2.columns)

In [ ]:
X_embedded = df_embedded
X_non_embedded = data2

# XGBoost

# Grid Search for Outcome Model

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer, mean_squared_error



xgb_gridsearch = XGBRegressor(objective="reg:squarederror", random_state=42, tree_method='hist', device='cuda')



X = df_none
y = y

# Define the parameter grid
# Define the hyperparameter grid including min_child_weight
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'min_child_weight': [1, 5, 10],
}

# Define the cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric (negative MSE for minimization)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform grid search
grid_search = GridSearchCV(
    estimator=xgb_gridsearch,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    verbose=1,
    n_jobs=-1  # Parallelize
)

# Fit grid search
grid_search.fit(X, y)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Negative MSE):", grid_search.best_score_)


Fitting 5 folds for each of 250 candidates, totalling 1250 fits


# Grid Search for Treatment Model

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.metrics import log_loss


# Example treatment data
X = df_none  # Feature matrix
T = T         # Treatment vector (binary)

scorer_continuous = make_scorer(mean_squared_error, greater_is_better=False)

xgb_treatment_model = XGBRegressor(objective="reg:squarederror", random_state=42, tree_method='hist', device='cuda')

# Define the parameter grid
param_grid_treatment = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'min_child_weight': [1, 5, 10],
}

# Define the cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metric (log loss for classification)
scorer = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# Perform grid search
grid_search_treatment = GridSearchCV(
    estimator=xgb_treatment_model,
    param_grid=param_grid_treatment,
    scoring=scorer,
    cv=cv,
    verbose=1,
    n_jobs=-1  # Parallelize
)

# Fit grid search
grid_search_treatment.fit(X, T)

# Print best parameters and score
print("Best Parameters for Treatment Model:", grid_search_treatment.best_params_)
print("Best Score (Negative Log Loss):", grid_search_treatment.best_score_)